# Week 3 part 3 assignment

In [2]:
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
! pip install geocoder
import geocoder

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/zettadevs/anaconda3

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-package-handling-1.4.1|           py37_0         262 KB  conda-forge
    -------------------------------------------------------

In [3]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [9]:
TorontoData  = pd.read_csv('task2.csv')
TorontoData .head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [6]:
location = get_latlng('M4G')
location

[43.70976500000006, -79.36390090899994]

In [7]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = TorontoData['Latitude']
Y = TorontoData['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
TorontoData['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(TorontoData['Latitude'], TorontoData['Longitude'], TorontoData['Borough'], TorontoData['Cluster']):
    label = folium.Popup(borough, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

In [10]:
## Define Foursquare Credentials and Version
CLIENT_ID = 'I0VGPUVOBBKZVPRYS0GTZDVDG01DPS0N4VY0MR2TAB1KEN2S' # your Foursquare ID
CLIENT_SECRET = 'X04GZ4TJ2EGXNMMPJYUPJCXZIMLJGBV1CUUNXRHYM22ECON3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: I0VGPUVOBBKZVPRYS0GTZDVDG01DPS0N4VY0MR2TAB1KEN2S
CLIENT_SECRET:X04GZ4TJ2EGXNMMPJYUPJCXZIMLJGBV1CUUNXRHYM22ECON3


In [13]:
TorontoData[TorontoData['Neighbourhood']=='Christie'].index

Int64Index([75], dtype='int64')

In [15]:
neighborhood_latitude = TorontoData.loc[75, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = TorontoData.loc[75, 'Longitude'] # neighborhood longitude value

neighborhood_name = TorontoData.loc[75, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Christie are 43.66878132800008, -79.42070999999999.


In [16]:
LIMIT = 150
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=I0VGPUVOBBKZVPRYS0GTZDVDG01DPS0N4VY0MR2TAB1KEN2S&client_secret=X04GZ4TJ2EGXNMMPJYUPJCXZIMLJGBV1CUUNXRHYM22ECON3&ll=43.71174000000008,-79.57918134599998&v=20180605&radius=500&limit=150'

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d7d5d5e2c20170036563171'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Toronto',
  'headerFullLocation': 'Toronto',
  'headerLocationGranularity': 'city',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.71624000450008,
    'lng': -79.57296739525327},
   'sw': {'lat': 43.707239995500075, 'lng': -79.5853952967467}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b60bccff964a5209cf629e3',
       'name': "Harvey's",
       'location': {'address': '269 Rexdale Blvd.',
        'lat': 43.71228923363633,
        'lng': -79.57743591075904,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.71228923363633,
          'ln

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Harvey's,Burger Joint,43.712289,-79.577436
1,Tim Hortons,Coffee Shop,43.713177,-79.580186
2,Subway,Sandwich Place,43.713345,-79.580358
3,U-Haul Moving & Storage at Rexdale Blvd,Storage Facility,43.713500,-79.577307
4,Fitness Two 4,Gym,43.714482,-79.582903


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

6 venues were returned by Foursquare.


In [22]:
## create a function to repeat the same process to all the neighborhoods in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
# find the venues in all Toronto Neighbohoods
toronto_venues = getNearbyVenues(names=TorontoData['Neighbourhood'],
                                   latitudes=TorontoData['Latitude'],
                                   longitudes=TorontoData['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [26]:
print(toronto_venues.shape)
toronto_venues.head()

(2474, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,Royal Canadian Legion,43.782533,-79.163085,Bar
1,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping
2,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Heron Park Community Centre,43.768867,-79.176958,Gym / Fitness Center
3,"Guildwood,Morningside,West Hill",43.765690,-79.175256,Heron Park,43.769327,-79.177201,Park
4,Woburn,43.768359,-79.217590,Starbucks,43.770037,-79.221156,Coffee Shop


In [27]:
## check how many venues were returned for each neighborhood
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,14,14,14,14,14,14
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",2,2,2,2,2,2
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",14,14,14,14,14,14
"Alderwood,Long Branch",4,4,4,4,4,4
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",20,20,20,20,20,20
Berczy Park,62,62,62,62,62,62
"Birch Cliff,Cliffside West",6,6,6,6,6,6


In [28]:
## find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 255 uniques categories.


In [29]:
## Analyze Each Neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
column_list = toronto_onehot.columns.tolist()
column_number = int(column_list.index('Neighbourhood'))
column_list = [column_list[column_number]] + column_list[:column_number] + column_list[column_number+1:]
toronto_onehot = toronto_onehot[column_list]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Transportation Service,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Highland Creek,Rouge Hill,Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood,Morningside,West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Woburn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
toronto_onehot.shape

(2474, 256)

In [31]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Transportation Service,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.030000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.0,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.000000,0.0,0.000000
4,"Alderwood,Long Branch",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Bedford Park,Lawrence Manor East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.016129,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,"Birch Cliff,Cliffside West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [32]:
toronto_grouped.shape

(99, 256)

In [34]:
## print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.09
1         Café  0.06
2        Hotel  0.04
3   Restaurant  0.03
4          Bar  0.03


----Agincourt----
                 venue  freq
0        Shopping Mall  0.14
1   Chinese Restaurant  0.14
2          Supermarket  0.14
3         Skating Rink  0.07
4  Shanghai Restaurant  0.07


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                   Pharmacy   1.0
1          Afghan Restaurant   0.0
2                  Nightclub   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0         Grocery Store  0.14
1           Coffee Shop  0.07
2  Fast Food Restaurant  0.07
3          Liquor Store  0.07
4           Pizza Place  0.07


----Alderwood,Long Branch----
               venue  freq
0  Convenience Store  0

                 venue  freq
0          Coffee Shop  0.13
1                 Café  0.07
2                Hotel  0.07
3           Restaurant  0.04
4  American Restaurant  0.03


----Flemingdon Park,Don Mills South----
           venue  freq
0  Grocery Store  0.14
1   Intersection  0.14
2            Gym  0.14
3    Coffee Shop  0.14
4     Beer Store  0.14


----Forest Hill North,Forest Hill West----
                       venue  freq
0                       Park   1.0
1          Afghan Restaurant   0.0
2                  Nightclub   0.0
3         Mexican Restaurant   0.0
4  Middle Eastern Restaurant   0.0


----Glencairn----
                      venue  freq
0               Pizza Place  0.27
1      Fast Food Restaurant  0.13
2  Mediterranean Restaurant  0.07
3        Italian Restaurant  0.07
4          Sushi Restaurant  0.07


----Guildwood,Morningside,West Hill----
                        venue  freq
0  Construction & Landscaping  0.33
1        Gym / Fitness Center  0.33
2                

                venue  freq
0                Café  0.11
1      Sandwich Place  0.11
2         Pizza Place  0.07
3         Coffee Shop  0.07
4  Mexican Restaurant  0.04


----The Beaches----
               venue  freq
0       Neighborhood  0.25
1                Pub  0.25
2              Trail  0.25
3  Health Food Store  0.25
4  Afghan Restaurant  0.00


----The Beaches West,India Bazaar----
               venue  freq
0     Sandwich Place  0.11
1               Park  0.11
2     Ice Cream Shop  0.05
3       Burger Joint  0.05
4  Fish & Chips Shop  0.05


----The Danforth West,Riverdale----
                       venue  freq
0                   Bus Line  0.25
1                       Park  0.25
2             Discount Store  0.25
3              Grocery Store  0.25
4  Middle Eastern Restaurant  0.00


----The Junction North,Runnymede----
                    venue  freq
0                 Brewery  0.43
1                    Park  0.14
2  Furniture / Home Store  0.14
3            Home Service  0.14

In [35]:
## put that into a pandas dataframe
## First, write a function to sort the venues in descending order.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
## create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Seafood Restaurant,Gym,Burger Joint,American Restaurant,Japanese Restaurant,Bar,Restaurant
1,Agincourt,Supermarket,Shopping Mall,Chinese Restaurant,Hong Kong Restaurant,Vietnamese Restaurant,Skating Rink,Shanghai Restaurant,Sushi Restaurant,Pool,Bakery
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Pharmacy,Yoga Studio,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Park,Coffee Shop,Japanese Restaurant,Sandwich Place,Fried Chicken Joint,Liquor Store,Hardware Store,Video Store
4,"Alderwood,Long Branch",Convenience Store,Pub,Gym,Sandwich Place,Fish & Chips Shop,Field,Fast Food Restaurant,Fish Market,Dumpling Restaurant,Farm
5,Bayview Village,Dog Run,Construction & Landscaping,Park,Trail,Farmers Market,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
6,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Thai Restaurant,Butcher,Café,Liquor Store,Sandwich Place,Sports Club,Restaurant,Sushi Restaurant
7,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Bakery,Breakfast Spot,Cheese Shop,Café,Seafood Restaurant,Beer Bar,Farmers Market
8,"Birch Cliff,Cliffside West",Skating Rink,College Stadium,Gym,Gym Pool,Park,General Entertainment,Farmers Market,Fast Food Restaurant,Field,Dumpling Restaurant
9,"Bloordale Gardens,Eringate,Markland Wood,Old B...",College Rec Center,Bank,Grocery Store,Fish & Chips Shop,Carpet Store,Shopping Mall,Electronics Store,Farm,Event Space,Falafel Restaurant


In [48]:
## Cluster Neighborhoods
## Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
kmeans.labels_.shape

(99,)

In [50]:
# add clustering labels
toronto_merged = TorontoData.head(99)
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head()

/home/zettadevs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750,1,Bar,Yoga Studio,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256,1,Park,Construction & Landscaping,Gym / Fitness Center,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
3,M1G,Scarborough,Woburn,43.768359,-79.217590,1,Korean Restaurant,Coffee Shop,Business Service,Park,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,1,Playground,Hong Kong Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market


In [58]:
# create final map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]
colors = ['red', 'green', 'blue', 'yellow','orange']

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color=colors[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [59]:
## Examine Clusters

## Cluster 1
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,0,Train Station,Restaurant,Grocery Store,Indian Restaurant,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
8,Scarborough,0,Fast Food Restaurant,Burger Joint,Furniture / Home Store,Pharmacy,Discount Store,Coffee Shop,Sandwich Place,Liquor Store,Pizza Place,Gym Pool
15,Scarborough,0,Chinese Restaurant,Fast Food Restaurant,Coffee Shop,Grocery Store,Noodle House,Sandwich Place,Pizza Place,Other Great Outdoors,Pharmacy,Farmers Market
28,North York,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,North York,0,Park,Moving Target,Business Service,Health & Beauty Service,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
35,East York,0,Pizza Place,Fast Food Restaurant,Intersection,Pet Store,Rock Climbing Spot,Café,Pharmacy,Breakfast Spot,Gastropub,Bank
37,East Toronto,0,Trail,Health Food Store,Pub,Neighborhood,Yoga Studio,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
38,East York,0,Sporting Goods Shop,Sports Bar,Coffee Shop,Furniture / Home Store,Brewery,Sandwich Place,Sushi Restaurant,Supermarket,Mexican Restaurant,Breakfast Spot
40,East York,0,Bar,Farmers Market,Italian Restaurant,Park,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
41,East Toronto,0,Bus Line,Grocery Store,Park,Discount Store,Yoga Studio,Farmers Market,Event Space,Falafel Restaurant,Farm,Field


In [60]:
## Cluster 2
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Scarborough,1,Bar,Yoga Studio,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant
2,Scarborough,1,Park,Construction & Landscaping,Gym / Fitness Center,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Yoga Studio
3,Scarborough,1,Korean Restaurant,Coffee Shop,Business Service,Park,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Yoga Studio
4,Scarborough,1,Playground,Hong Kong Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant,Farmers Market
6,Scarborough,1,Discount Store,Convenience Store,Hobby Shop,Coffee Shop,Department Store,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
7,Scarborough,1,Coffee Shop,Intersection,Bus Line,Bakery,Soccer Field,Metro Station,Bus Station,Dim Sum Restaurant,Falafel Restaurant,Dance Studio
9,Scarborough,1,Skating Rink,College Stadium,Gym,Gym Pool,Park,General Entertainment,Farmers Market,Fast Food Restaurant,Field,Dumpling Restaurant
10,Scarborough,1,Bakery,Gift Shop,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Eastern European Restaurant
11,Scarborough,1,Convenience Store,Auto Garage,Eastern European Restaurant,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Field,Fast Food Restaurant


In [61]:
## Cluster 3
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,2,Dog Run,Residential Building (Apartment / Condo),Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Eastern European Restaurant


In [62]:
## Cluster 4
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Central Toronto,3,Breakfast Spot,Park,Food & Drink Shop,Gym,Clothing Store,Hotel,Fish Market,Fish & Chips Shop,Field,Flea Market
51,Downtown Toronto,3,Coffee Shop,Restaurant,Pet Store,Italian Restaurant,Bakery,Pizza Place,Market,Chinese Restaurant,Café,Farm


In [63]:
## Cluster 5
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
66,Downtown Toronto,4,Café,Restaurant,Bakery,Coffee Shop,Bookstore,Gym,Japanese Restaurant,Bar,College Gym,Burrito Place
